In [ ]:
import numpy as np 
import pandas as pd 
import os
from torchvision import transforms, utils, models, datasets
import torch
from torch.utils.data import DataLoader
import torchvision.transforms.functional as F

from utils.engine import train_one_epoch, evaluate
from utils import utils

import torch
from utils import transforms as T

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

from ADEdataset import ADEDataset

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

compose = transforms.Compose([transforms.PILToTensor(),
                                transforms.ConvertImageDtype(torch.float),
                                transforms.RandomHorizontalFlip(0.5),])
                                #transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])


def voc_transform_ing(img):
    
    target_anno = img["annotation"]
    w, h, d = list(map(int, target_anno['size'].values()))

    target = {"boxes":[], "labels":[],'area':[]}
    for obj in target_anno['object']:
        box = list(map(int,obj['bndbox'].values()))
        target["boxes"].append(box)
        target['labels'].append(classes.index(obj["name"]))
        
        area = (box[3] - box[1]) * (box[2] - box[0])
        target["area"].append(area)    

    iscrowd = torch.zeros((len(target["boxes"]),), dtype=torch.int64)

    target["image_id"] = torch.Tensor([int(target_anno['filename'][:-4])])
    target["boxes"] = torch.FloatTensor(target["boxes"])
    target["labels"] = torch.LongTensor(target["labels"])
    target["area"] = torch.FloatTensor(target["area"])
    target["iscrowd"] = iscrowd
    target["masks"] = torch.zeros((len(target["boxes"]),w, h), dtype=torch.float32)

    return target

In [ ]:
classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 
           'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
           'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']


path_dir_trainval = "../hw4_dataset/VOCTrainVal"
path_dir_test = "../hw4_dataset/VOCtest"

voctrainval_ds = datasets.VOCDetection(root=path_dir_trainval, year="2007", image_set="trainval", transform=compose, target_transform=voc_transform_ing)
voc_trainval = DataLoader(voctrainval_ds, batch_size=8, collate_fn = utils.collate_fn, shuffle=True)

voctest_ds = datasets.VOCDetection(root=path_dir_test, year="2007", image_set="test", transform=compose,target_transform=voc_transform_ing)
voc_test = DataLoader(voctest_ds, batch_size=8, collate_fn = utils.collate_fn, shuffle=True)


In [ ]:
print(f"trainval dataset :{len(voctrainval_ds)} images")
print(f"test dataset :{len(voctest_ds)} images")
print(f"the meta data in image:{voctrainval_ds[0][1].keys()}")

In [ ]:
root = "../ADE20K"
train_data = ADEDataset(root,"train.txt", get_transform(train=True))
ade_train = torch.utils.data.DataLoader(
                                train_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)
val_data = ADEDataset(root,"val.txt",  get_transform(train=True))
ade_val = torch.utils.data.DataLoader(
                                val_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)
test_data = ADEDataset(root,"test.txt",  get_transform(train=True))
ade_test = torch.utils.data.DataLoader(
                                test_data, batch_size=2, shuffle=True, num_workers=4,
                                collate_fn=utils.collate_fn)


In [ ]:
print(f"train dataset :{len(train_data)} images")
print(f"val dataset :{len(val_data)} images")
print(f"test dataset :{len(test_data)} images")
# print(f"the meta data in image:{voctrainval_ds[0][1].keys()}")

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

lr = 0.001
batch_size = 8
weight_decay=1e-5
num_classes = 150
model = get_model_instance_segmentation(num_classes).to(device)
#model = models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(score_thresh=0.5, weights_backbone=True, num_classes=num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr = lr, weight_decay=weight_decay)

In [ ]:
# voc_trainval, voc_test, ade_train, ade_val, ade_test

In [ ]:
num_epochs = 1

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
round = ["voc",'ade']
for epoch in range(num_epochs):
    
    train_one_epoch(model, optimizer, ade_train, device, epoch=epoch, run_dataset='ade', 
                    print_freq=10, scaler=None)
    
    train_one_epoch(model, optimizer, voc_trainval, device, epoch=epoch, run_dataset='voc', 
                 print_freq=10, scaler=None)
    
    
    lr_scheduler.step()
        # evaluate on the test dataset3
    evaluate(model, voc_test, device=device)
    evaluate(model, ade_test, device=device)